In [2]:
import psycopg2
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
import re

host = 'aact-db.ctti-clinicaltrials.org'
port = '5432'
user = ''
password = ''
database = 'aact'

conn_str = "host={} dbname={} user={} password={}".format(host, database, user, password)
conn = psycopg2.connect(conn_str)
print(conn_str.split())

['host=aact-db.ctti-clinicaltrials.org', 'dbname=aact', 'user=csyuming', 'password=19911030']


### Extract Data from SQL

In [2]:
# Alzheimer related clinical trials
sql = """SELECT stud.*, a.name AS sponsor_name, a.agency_class, i.name AS drug_name , i.intervention_type
            FROM studies stud 
            INNER JOIN (SELECT * FROM sponsors WHERE lead_or_collaborator = 'lead') a ON a.nct_id = stud.nct_id 
            INNER JOIN (SELECT DISTINCT name, nct_id, intervention_type FROM interventions) i ON i.nct_id = stud.nct_id 
            WHERE stud.nct_id IN (SELECT nct_id FROM browse_conditions WHERE downcase_mesh_term LIKE '%alzheimer%')
            AND stud.study_type = 'Interventional'
            AND (intervention_type = 'Drug')
            AND stud.nct_id IN (SELECT nct_id FROM designs WHERE primary_purpose = 'Treatment' AND outcomes_assessor_masked = 'true' AND masking != 'None (Open Label)')
            """
df=pd.read_sql(sql, con=conn)

In [5]:
# # save to local drive
# df.to_csv('Alzheimer_Clinical_Trails.csv', index=False)

In [4]:
df = pd.read_csv('./Alzheimer_Clinical_Trails.csv')
df.intervention_type.value_counts()

Drug    598
Name: intervention_type, dtype: int64

In [5]:
df1 = df.copy()

### Explorary Data Analysis

Causes of duplicate ***nct_id*** in joined table
- nct_id is primary key, it is unique in ***studies***
- only one ***lead*** sponsor associate on nct_id
- duplicate nct_id in the joined table is caused by ***one-to-many*** relationship between ***studies*** and ***interventions***  

We are going to try to clean the format of drug name next. Many contain dosage in the name.

In [6]:
# duplicate nut_id
df1['nct_id'].value_counts()[df1['nct_id'].value_counts()>1]

NCT00766363    6
NCT02051335    5
NCT03001557    5
NCT00948766    4
NCT00439166    4
              ..
NCT02925650    2
NCT00842816    2
NCT02291783    2
NCT03625622    2
NCT01584440    2
Name: nct_id, Length: 235, dtype: int64

#### Clean Drug Names Contain ***'mg/'*** ####

In [7]:
# clean drug name contains 'mg/day'
ix = df1[df1['drug_name'].str.contains('mg/')].index
loc = [i.find(' ') for i in df1[df1['drug_name'].str.contains('mg/')]['drug_name']]

for i,l in zip(ix,loc):
    df1.loc[i, 'drug_name'] = df1.loc[i, 'drug_name'][:l]

In [8]:
# double check 
df1[df1['drug_name'].str.contains('mg/')]

,nct_id,nlm_download_date_description,study_first_submitted_date,results_first_submitted_date,disposition_first_submitted_date,last_update_submitted_date,study_first_submitted_qc_date,study_first_posted_date,study_first_posted_date_type,results_first_submitted_qc_date,results_first_posted_date,results_first_posted_date_type,disposition_first_submitted_qc_date,disposition_first_posted_date,disposition_first_posted_date_type,last_update_submitted_qc_date,last_update_posted_date,last_update_posted_date_type,start_month_year,start_date_type,start_date,verification_month_year,verification_date,completion_month_year,completion_date_type,completion_date,primary_completion_month_year,primary_completion_date_type,primary_completion_date,target_duration,study_type,acronym,baseline_population,brief_title,official_title,overall_status,last_known_status,phase,enrollment,enrollment_type,source,limitations_and_caveats,number_of_arms,number_of_groups,why_stopped,has_expanded_access,expanded_access_type_individual,expanded_access_type_intermediate,expanded_access_type_treatment,has_dmc,is_fda_regulated_drug,is_fda_regulated_device,is_unapproved_device,is_ppsd,is_us_export,biospec_retention,biospec_description,ipd_time_frame,ipd_access_criteria,ipd_url,plan_to_share_ipd,plan_to_share_ipd_description,created_at,updated_at,sponsor_name,agency_class,drug_name,intervention_type


Now, after modified drug names there should be some duplicates. We are going to drop thoes duplicates

In [9]:
len(df1) - len(df1.drop_duplicates())

6

We are going to drop these rows have the same drug name

In [10]:
df1.drop_duplicates(inplace=True)

#### Clean Drug Names Contain mg

- Start with drug_name has 2 letters
- locate ' '(blanck space), if a space after mg the drug name would later words; if no space after mg then drug name would be first word

Find rows have ***one space*** after 'mg' in drug name, example: 15mg T3D-959

In [11]:
# rows has two words in drug name and contain 'mg'
mg_one_space = df1[(df1['drug_name'].str.count(' ') == 1) & (df1['drug_name'].str.contains('mg'))]

# find rows have a space after mg
mg_one_space_after = mg_one_space[mg_one_space['drug_name'].str.contains('mg ')]
mg_one_space_after['drug_name']

0    15mg T3D-959
Name: drug_name, dtype: object

In [12]:
# for loop to change the name one by one
for ix in mg_one_space_after.index:
    old_name = mg_one_space_after.loc[ix, 'drug_name']
    space_location = old_name.find(' ')
    # plus 1 to add space location
    new_name = old_name[space_location + 1:]
    #assign new name
    mg_one_space.loc[ix, 'drug_name'] = new_name
    mg_one_space_after.loc[ix, 'drug_name'] = new_name
    df1.loc[ix, 'drug_name'] = new_name

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
# double check
mg_one_space_after['drug_name']

0    T3D-959
Name: drug_name, dtype: object

Find rows have a space before 'mg', example :'Azeliragon 5mg'

In [14]:
# since we already clean the drug name has dosage after, the reset have dosage before space
mg_one_space_before = df1[(df1['drug_name'].str.count(' ') == 1) & (df1['drug_name'].str.contains('mg'))]
mg_one_space_before['drug_name']

94     CNP520 15mg
95     CNP520 50mg
542    S47445 15mg
543    S47445 50mg
544     S47445 5mg
Name: drug_name, dtype: object

In [15]:
# for loop to change the name one by one
for ix in mg_one_space_before.index:
    old_name = mg_one_space_before.loc[ix, 'drug_name']
    space_location = old_name.find(' ')
    # plus 1 to add space location
    new_name = old_name[:space_location]
    #assign new name
    mg_one_space.loc[ix, 'drug_name'] = new_name
    mg_one_space_before.loc[ix, 'drug_name'] = new_name
    df1.loc[ix, 'drug_name'] = new_name

In [16]:
mg_one_space_before['drug_name']

94     CNP520
95     CNP520
542    S47445
543    S47445
544    S47445
Name: drug_name, dtype: object

Clean drug names have ***two spaces*** and have dosage in it
- ***1st case***: 'mg' is in between example: 30 mg T3D-959
- ***2nd case***: 'mg' at the end example: Atabecestat, 25 mg
- ***3rd case***: there is no space between dosage amount and unit 'mg' example: AD-35 90mg group

In [17]:
# rows has three words in drug name and contain 'mg'
mg_two_space = df1[(df1['drug_name'].str.count(' ') == 2) & (df1['drug_name'].str.contains('mg'))]
# number of rows
mg_two_space['drug_name'].head()

4            30 mg T3D-959
6            45 mg T3D-959
44     AZD0530 100mg daily
45     AZD0530 125mg daily
187         GV1001 0.56 mg
Name: drug_name, dtype: object

In [18]:
# function to split dosage for the first case
def split_dosage(drug_name):
    # space before and after 'mg' identify the first case
    try:
        return re.split(r" mg ", drug_name)[1]
    except:
        return drug_name

In [19]:
# apply the function to drug_name column
mg_two_space['drug_name'] = mg_two_space['drug_name'].apply(split_dosage)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
print('Orginal name: ' + mg_two_space.loc[187, 'drug_name']+
      '\nAfter Regex split: '+re.split(r"\s\d+\.?\d*\smg$", mg_two_space.loc[187, 'drug_name'])[0])

Orginal name: GV1001 0.56 mg
After Regex split: GV1001


In [21]:
# function to split dosage for the second case
def split_dosage(drug_name):
    # space before and after 'mg' identify the sencond case
    try:
        return re.split(r"\s\d+\.?\d*\smg$", drug_name)[0]
    except:
        return drug_name

In [22]:
# apply the function to drug_name column
mg_two_space['drug_name'] = mg_two_space['drug_name'].apply(split_dosage)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
mg_two_space['drug_name']

4                  T3D-959
6                  T3D-959
44     AZD0530 100mg daily
45     AZD0530 125mg daily
187                 GV1001
188                 GV1001
210            Lemborexant
211            Lemborexant
212            Lemborexant
213            Lemborexant
292           Pimavanserin
293           Pimavanserin
541                RVT-101
Name: drug_name, dtype: object

Looks like it works with majority drug names except few don't have space between dosage amount and unit example: ***AD-35 60mg group***. Simple Regex split would work for these

In [24]:
print('Orginal name: ' + mg_two_space.loc[44, 'drug_name']+
      '\nAfter Regex split: '+re.split(r"\s\d+mg", mg_two_space.loc[44, 'drug_name'])[0])

Orginal name: AZD0530 100mg daily
After Regex split: AZD0530


In [25]:
# function to split dosage for the second case
def split_dosage(drug_name):
    # space before and after 'mg' identify the sencond case
    try:
        return re.split(r"\s\d+mg", drug_name)[0]
    except:
        return drug_name
# apply the function to drug_name column
mg_two_space['drug_name'] = mg_two_space['drug_name'].apply(split_dosage)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [26]:
mg_two_space['drug_name']

4           T3D-959
6           T3D-959
44          AZD0530
45          AZD0530
187          GV1001
188          GV1001
210     Lemborexant
211     Lemborexant
212     Lemborexant
213     Lemborexant
292    Pimavanserin
293    Pimavanserin
541         RVT-101
Name: drug_name, dtype: object

In [27]:
# replace the new name into our df1 dataframe
df1.loc[mg_two_space.index, 'drug_name'] = mg_two_space['drug_name']

Next we are going to look up drug names contain ***three spaces***. The names are geting more compalicated with more spaces involved

In [28]:
# rows has three words in drug name and contain 'mg'
mg_three_space = df1[(df1['drug_name'].str.count(' ') == 3) & (df1['drug_name'].str.contains('mg'))]
# number of rows
mg_three_space['drug_name']

27                   AGB101 220 mg tablet
516                  PTI-125 50 mg tablet
534    Rosiglitazone Extended Release 2mg
535    Rosiglitazone Extended Release 2mg
536    Rosiglitazone Extended Release 8mg
537    Rosiglitazone Extended Release 8mg
Name: drug_name, dtype: object

In [29]:
print('Old name: '+mg_three_space.loc[534, 'drug_name'] +
      '\nNew name: ' + re.split(r'\s\d+\.?\-?\d*\s*mg', mg_three_space.loc[534, 'drug_name'])[0])

Old name: Rosiglitazone Extended Release 2mg
New name: Rosiglitazone Extended Release


In [30]:
# function to split dosage for the second case
def split_dosage(drug_name):
    # space before and after 'mg' identify the sencond case
    try:
        return re.split(r'\s\d+\.?\-?\d*\s*mg', drug_name)[0]
    except:
        return drug_name
mg_three_space['drug_name'] = mg_three_space['drug_name'].apply(split_dosage)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
mg_three_space.drug_name

27                             AGB101
516                           PTI-125
534    Rosiglitazone Extended Release
535    Rosiglitazone Extended Release
536    Rosiglitazone Extended Release
537    Rosiglitazone Extended Release
Name: drug_name, dtype: object

We have one more need to correct, it use ***milligram*** instead of ***mg***

In [32]:
# function to split dosage for the second case
def split_dosage(drug_name):
    # space before and after 'mg' identify the sencond case
    try:
        return re.split(r'\s\d+\.?\d*\smilligram\s', drug_name)[0]
    except:
        return drug_name
mg_three_space['drug_name'] = mg_three_space['drug_name'].apply(split_dosage)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
mg_three_space.drug_name

27                             AGB101
516                           PTI-125
534    Rosiglitazone Extended Release
535    Rosiglitazone Extended Release
536    Rosiglitazone Extended Release
537    Rosiglitazone Extended Release
Name: drug_name, dtype: object

In [34]:
# replace the drug name to main dataframe
df1.loc[mg_three_space.index, 'drug_name'] = mg_three_space['drug_name']

All look good now, next we are going to work with drug names have more than three spaces

In [35]:
# rows has three words in drug name and contain 'mg'
mg_three_space_more = df1[(df1['drug_name'].str.count(' ') > 3) & (df1['drug_name'].str.contains('mg'))]
# number of rows
mg_three_space_more['drug_name']

22                     Active Treatment- CT1812 100 mg
23                     Active Treatment- CT1812 300 mg
477                    Placebo + Donepezil 5mg or 10mg
515                         PTI-125 100 mg oral tablet
585                 Verubecestat 12 mg (Parts 1 and 2)
586                 Verubecestat 40 mg (Parts 1 and 2)
596    Zydena (Udenafil) 100mg + Donepezil 5mg or 10mg
597     Zydena (Udenafil) 50mg + Donepezil 5mg or 10mg
Name: drug_name, dtype: object

In [36]:
# function to split dosage for the second case
def split_dosage(drug_name):
    # space before and after 'mg' identify the sencond case
    try:
        return re.split(r'\s\d+\.?\-?\d*\s*mg', drug_name)[0]
    except:
        return drug_name
mg_three_space_more['drug_name'] = mg_three_space_more['drug_name'].apply(split_dosage)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
# function to split dosage for the second case
def split_dosage(drug_name):
    # space before and after 'mg' identify the sencond case
    try:
        return re.split(r"\s\d+mg", drug_name)[0]
    except:
        return drug_name
# apply the function to drug_name column
mg_three_space_more['drug_name'] = mg_three_space_more['drug_name'].apply(split_dosage)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [38]:
# replace the drug name to main dataframe
df1.loc[mg_three_space_more.index, 'drug_name'] = mg_three_space_more['drug_name']

Since We have cleaned many drug names, now we may expect to see some duplicate rows, we are going to remove those rows

In [39]:
df1.drop_duplicates(inplace=True,)
df1.reset_index(drop=True, inplace=True)

We cleaned drug names have dosage related issue, next we'll remove clinical trails whice are designed for blined test

#### Clean Drug Names Contain Placebo ####

***placebo*** is a drug for blind test, one of test group will recieve ***placebo*** which has no active ingredients referred as control group. Another group will receive the treatmet to be evaluated.  * 

There are common Alzheimer drugs are used in the trails as standard of care, Some related trails are testing various investigational drugs as add-on therapy to donepezil. Some are testing efficacy, safety or tolerability of marketed drugs under certain conditions** 
 - donepezil
 - rivastigmine
 - galantamine
 - memantine  
*source: design_group of NCT01677754  
**source: https://www.alzforum.org/therapeutics/donepezil;  
https://www.alz.org/alzheimers-dementia/treatments/medications-for-memory

In [40]:
df1[df1['nct_id'] == 'NCT01255046']

,nct_id,nlm_download_date_description,study_first_submitted_date,results_first_submitted_date,disposition_first_submitted_date,last_update_submitted_date,study_first_submitted_qc_date,study_first_posted_date,study_first_posted_date_type,results_first_submitted_qc_date,results_first_posted_date,results_first_posted_date_type,disposition_first_submitted_qc_date,disposition_first_posted_date,disposition_first_posted_date_type,last_update_submitted_qc_date,last_update_posted_date,last_update_posted_date_type,start_month_year,start_date_type,start_date,verification_month_year,verification_date,completion_month_year,completion_date_type,completion_date,primary_completion_month_year,primary_completion_date_type,primary_completion_date,target_duration,study_type,acronym,baseline_population,brief_title,official_title,overall_status,last_known_status,phase,enrollment,enrollment_type,source,limitations_and_caveats,number_of_arms,number_of_groups,why_stopped,has_expanded_access,expanded_access_type_individual,expanded_access_type_intermediate,expanded_access_type_treatment,has_dmc,is_fda_regulated_drug,is_fda_regulated_device,is_unapproved_device,is_ppsd,is_us_export,biospec_retention,biospec_description,ipd_time_frame,ipd_access_criteria,ipd_url,plan_to_share_ipd,plan_to_share_ipd_description,created_at,updated_at,sponsor_name,agency_class,drug_name,intervention_type
129,NCT01255046,ClinicalTrials.gov processed this data on Janu...,2010-12-03,NaN,NaN,2014-08-19,2010-12-05,2010-12-07,Estimate,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-19,2014-08-20,Estimate,December 2015,NaN,2015-12-31,August 2014,2014-08-31,NaN,NaN,NaN,December 2018,Anticipated,2018-12-31,NaN,Interventional,NaN,NaN,Study of STA-1 as an Add-on Treatment to Donep...,"A Phase II Double-blind, Randomized, Placebo-c...",Unknown status,Not yet recruiting,Phase 2,136,Anticipated,"Sinphar Pharmaceutical Co., Ltd",NaN,2,NaN,NaN,False,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-01 16:35:48.924341,2020-02-01 16:35:48.924341,"Sinphar Pharmaceutical Co., Ltd",Other,"Donepezil,",Drug
301,NCT01255046,ClinicalTrials.gov processed this data on Janu...,2010-12-03,NaN,NaN,2014-08-19,2010-12-05,2010-12-07,Estimate,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-19,2014-08-20,Estimate,December 2015,NaN,2015-12-31,August 2014,2014-08-31,NaN,NaN,NaN,December 2018,Anticipated,2018-12-31,NaN,Interventional,NaN,NaN,Study of STA-1 as an Add-on Treatment to Donep...,"A Phase II Double-blind, Randomized, Placebo-c...",Unknown status,Not yet recruiting,Phase 2,136,Anticipated,"Sinphar Pharmaceutical Co., Ltd",NaN,2,NaN,NaN,False,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-01 16:35:48.924341,2020-02-01 16:35:48.924341,"Sinphar Pharmaceutical Co., Ltd",Other,placebo,Drug
544,NCT01255046,ClinicalTrials.gov processed this data on Janu...,2010-12-03,NaN,NaN,2014-08-19,2010-12-05,2010-12-07,Estimate,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-19,2014-08-20,Estimate,December 2015,NaN,2015-12-31,August 2014,2014-08-31,NaN,NaN,NaN,December 2018,Anticipated,2018-12-31,NaN,Interventional,NaN,NaN,Study of STA-1 as an Add-on Treatment to Donep...,"A Phase II Double-blind, Randomized, Placebo-c...",Unknown status,Not yet recruiting,Phase 2,136,Anticipated,"Sinphar Pharmaceutical Co., Ltd",NaN,2,NaN,NaN,False,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-01 16:35:48.924341,2020-02-01 16:35:48.924341,"Sinphar Pharmaceutical Co., Ltd",Other,STA-1,Drug


In [41]:
# example of add-on trails related to donepezil
[print(i) for i in df1[df1['nct_id'] == 'NCT01255046']['brief_title']]

Study of STA-1 as an Add-on Treatment to Donepezil
Study of STA-1 as an Add-on Treatment to Donepezil
Study of STA-1 as an Add-on Treatment to Donepezil


[None, None, None]

In [42]:
df1['nct_id'].value_counts()[df1['nct_id'].value_counts()>1]

NCT02051335    5
NCT01303744    4
NCT00866060    4
NCT01230853    4
NCT03752463    4
              ..
NCT00086138    2
NCT00842816    2
NCT01584440    2
NCT02925650    2
NCT03625622    2
Name: nct_id, Length: 230, dtype: int64

We still have 500+ duplicate nct_id

In [43]:
# lowercase drug name and find 'placebo'
ix = df1[df1['drug_name'].str.lower().str.contains('placebo')].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [44]:
# lowercase drug name and find 'placebo'
ix = df1[df1['drug_name'].str.lower().str.contains('donepezil')].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [45]:
# lowercase drug name and find 'placebo'
ix = df1[df1['drug_name'].str.lower().str.contains('rivastigmine')].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [46]:
# lowercase drug name and find 'placebo'
ix = df1[df1['drug_name'].str.lower().str.contains('memantine')].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [47]:
# lowercase drug name and find 'placebo'
ix = df1[df1['drug_name'].str.lower().str.contains('galantamine')].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [48]:
# lowercase drug name and find 'placebo'
ix = df1[df1['drug_name'].str.lower().str.contains('dimebon')].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [49]:
# lowercase drug name and find 'placebo'
ix = df1[df1['drug_name'].str.lower().str.contains('aricept')].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [50]:
df1['nct_id'].value_counts()[df1['nct_id'].value_counts()>1]

NCT02168920    3
NCT02380573    3
NCT03752463    3
NCT00955409    3
NCT01595646    3
NCT01203384    3
NCT01303744    3
NCT00439166    2
NCT00056329    2
NCT01119638    2
NCT03867253    2
NCT01614886    2
NCT01766336    2
NCT01436045    2
NCT03289143    2
NCT02547818    2
NCT01760005    2
NCT03790709    2
NCT00996918    2
NCT01230853    2
NCT04052737    2
NCT02667496    2
NCT03959553    2
NCT02079909    2
NCT01469351    2
NCT00575055    2
NCT00141661    2
NCT01211782    2
NCT01608217    2
NCT01569516    2
NCT03184467    2
NCT03828747    2
NCT00066157    2
NCT02051335    2
NCT00574132    2
NCT04044131    2
Name: nct_id, dtype: int64

After removing rows contain 'placebo', we have 100+ duplicate nct_id

#### Remove Vitamin Related Trials

In [51]:
# drop these vitamin related trials
ix = df1[df1['drug_name'].str.lower().str.contains('vitamin', na=False)].index
df1.drop(ix, inplace=True)
df1.reset_index(inplace=True, drop=True)

In [52]:
df1['nct_id'].value_counts()[df1['nct_id'].value_counts()>1]

NCT02380573    3
NCT01303744    3
NCT00955409    3
NCT03752463    3
NCT01203384    3
NCT01595646    3
NCT02168920    3
NCT03959553    2
NCT03289143    2
NCT03867253    2
NCT01614886    2
NCT01766336    2
NCT00439166    2
NCT01436045    2
NCT01760005    2
NCT02547818    2
NCT00996918    2
NCT01230853    2
NCT04052737    2
NCT03790709    2
NCT01119638    2
NCT02667496    2
NCT02079909    2
NCT00575055    2
NCT00066157    2
NCT01569516    2
NCT00141661    2
NCT03184467    2
NCT04044131    2
NCT02051335    2
NCT01211782    2
NCT01608217    2
NCT01469351    2
NCT03828747    2
NCT00574132    2
Name: nct_id, dtype: int64

#### Clean Minor Differences With One Space

There are many minor adjustments we need to do for the drug names, we are going to scale the list down to ***industry(agency_class)*** only.

In [53]:
# agency_class industry only
df1_industry = df1[df1['agency_class']=='Industry']

In [54]:
# there are about 
print('The number of compnies in the list: {}'.format(len(df1_industry['sponsor_name'].unique())))

The number of compnies in the list: 85


Clean the rows with one space in drug name

If rows has the same nct_id, split the drug name into two parts and use the most common part as the drug name

In [55]:
# get index for drug names have one space
nct_id_count = df1_industry[df1_industry['drug_name'].str.count(' ') == 1]['nct_id'].value_counts()
nct_id_count[nct_id_count>1]

NCT01203384    3
NCT01211782    2
NCT02667496    2
Name: nct_id, dtype: int64

In [56]:
trial_id = nct_id_count[nct_id_count>1].index

In [57]:
[print(i, df1_industry[df1_industry['nct_id'] == i]['drug_name'].values) for i in trial_id]

NCT01203384 ['CHF5074 1x' 'CHF5074 2x' 'CHF5074 3x']
NCT01211782 ['caprylic triglyceride' 'long-chain triglyceride']
NCT02667496 ['Florbetapir F18' 'Sargramostim GZ402664']


[None, None, None]

In [58]:
# manually correct some
df1_industry.loc[df1_industry['nct_id'] == 'NCT01203384', 'drug_name'] = 'CHF5074'
df1_industry.loc[df1_industry['nct_id'] == 'NCT01258452', 'drug_name'] = 'CHF5074'
df1_industry.loc[df1_industry['nct_id'] == 'NCT02667496', 'drug_name'] = 'sargramostim'
df1_industry.loc[df1_industry['nct_id'] == 'NCT01211782', 'drug_name'] = 'triglyceride'
df1_industry.loc[df1_industry['nct_id'] == 'NCT02244541', 'drug_name'] = 'ANAVEX2-73'


In [59]:
# get index for drug names have more spaces
nct_id_count = df1_industry[df1_industry['drug_name'].str.count(' ') > 1]['nct_id'].value_counts()
nct_id_count[nct_id_count>1]

NCT01303744    3
NCT00141661    2
NCT01766336    2
NCT01569516    2
NCT04052737    2
NCT01230853    2
NCT03867253    2
NCT03790709    2
Name: nct_id, dtype: int64

In [60]:
trial_id = nct_id_count[nct_id_count>1].index
[print(i, df1_industry[df1_industry['nct_id'] == i]['drug_name'].values) for i in trial_id]

NCT01303744 ['CHF 5074 1x' 'CHF 5074 2x' 'CHF 5074 3x']
NCT00141661 ['PF-04494700 - High Dose Arm' 'PF-04494700 - Low Dose Arm']
NCT01766336 ['Group 1 ELND005' 'Group 2 ELND005']
NCT01569516 ['Octohydroaminoacridine succinate Tablets'
 'Octohydroaminoacridine Succinate Tablets']
NCT04052737 ['Normal Saline along with standard treatment'
 'PMZ-1620 (sovateltide) along with standard treatment']
NCT01230853 ['Active Comparator: A' 'Active Comparator B']
NCT03867253 ['ORY-2001 High dose' 'ORY-2001 Low dose']
NCT03790709 ['High dose ANAVEX2-73' 'Mid dose ANAVEX2-73']


[None, None, None, None, None, None, None, None]

In [61]:
# manually modify the name
df1_industry.loc[df1_industry['nct_id'] == 'NCT01303744', 'drug_name'] = 'CHF5074'
df1_industry.loc[df1_industry['nct_id'] == 'NCT04052737', 'drug_name'] = 'PMZ-1620'
df1_industry.loc[df1_industry['nct_id'] == 'NCT01569516', 'drug_name'] = 'Octohydroaminoacridine'
df1_industry.loc[df1_industry['nct_id'] == 'NCT00141661', 'drug_name'] = 'PF-04494700'
df1_industry.loc[df1_industry['nct_id'] == 'NCT01766336', 'drug_name'] = 'ELND005'
df1_industry.loc[df1_industry['nct_id'] == 'NCT01230853', 'drug_name'] = 'Active Comparator: A' # will be removed
df1_industry.loc[df1_industry['nct_id'] == 'NCT03790709', 'drug_name'] = 'ANAVEX2-73'
df1_industry.loc[df1_industry['nct_id'] == 'NCT03867253', 'drug_name'] = 'ORY-2001'

After correcting the name, there should be some duplicates, we are going drop those

In [62]:
print('We are going to drop {} rows'.format(len(df1_industry) - len(df1_industry.drop_duplicates())))

We are going to drop 13 rows


In [63]:
df1_industry.drop_duplicates(inplace=True)
df1_industry.reset_index(inplace=True, drop=True)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Finally, we want all the drug names have the same format, we are going downcase the name

In [64]:
df1_industry['drug_name'] = df1_industry['drug_name'].str.lower()

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Look Into Drugs by Each Company

In [65]:
print('There are {} drug companies have clinical trials related Alzheimer'.format(len(df1_industry['sponsor_name'].unique())))

There are 85 drug companies have clinical trials related Alzheimer


In [66]:
# companies have more than one clinical trials related Alzheimer
trials_more_than_one = df1_industry.groupby(['sponsor_name'])[['drug_name']].nunique()[df1_industry.groupby('sponsor_name')['drug_name'].nunique()>2]
trials_more_than_one.sort_values('drug_name',ascending=False)

,drug_name
sponsor_name,
Pfizer,11
Eisai Inc.,7
Eli Lilly and Company,5
AstraZeneca,4
"Otsuka Pharmaceutical Co., Ltd.",4
Biogen,3
Bristol-Myers Squibb,3
FORUM Pharmaceuticals Inc,3
"FUJIFILM Toyama Chemical Co., Ltd.",3


The process to correct name:
 - It looks like removing contents after space could be a way to correct name
 - Remove signs except '-' such as '+', ','
 - Double check the name only has letters
 - Pure numbers
 - length less than 3
 - words like drug, experimental, normal, active

In [67]:
# function to extract drug name
def name_split(drug_name):
    try:
        return re.split(r'\s', drug_name)[0]
    except:
        return drug_name

In [68]:
# fix name one by one
df1_industry.loc[(df1_industry['sponsor_name']=='GlaxoSmithKline'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='GlaxoSmithKline')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Pfizer'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Pfizer')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Wyeth is now a wholly owned subsidiary of Pfizer'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Wyeth is now a wholly owned subsidiary of Pfizer')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Merck Sharp & Dohme Corp.'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Merck Sharp & Dohme Corp.')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Eli Lilly and Company'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Eli Lilly and Company')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Eisai Inc.'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Eisai Inc.')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Sanofi'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Sanofi')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Hoffmann-La Roche'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Hoffmann-La Roche')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='AstraZeneca'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='AstraZeneca')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Biogen'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Biogen')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Otsuka Pharmaceutical Co., Ltd.'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Otsuka Pharmaceutical Co., Ltd.')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Novartis Pharmaceuticals'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Novartis Pharmaceuticals')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Otsuka Pharmaceutical Development & Commercialization, Inc.'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Otsuka Pharmaceutical Development & Commercialization, Inc.')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Bristol-Myers Squibb'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Bristol-Myers Squibb')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Cerecin'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Cerecin')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='JANSSEN Alzheimer Immunotherapy Research & Development, LLC'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='JANSSEN Alzheimer Immunotherapy Research & Development, LLC')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Avanir Pharmaceuticals'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Avanir Pharmaceuticals')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='GemVax & Kael'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='GemVax & Kael')]['drug_name'].apply(name_split)
df1_industry.loc[(df1_industry['sponsor_name']=='Eisai Co., Ltd.'), 'drug_name'] = df1_industry[(df1_industry['sponsor_name']=='Eisai Co., Ltd.')]['drug_name'].apply(name_split)


### Further Clean Base On Similarity

For this step, we are going to group the drug name by company and drop the duplicates
- There two rows have na, we are going to fillna as blank space
- Create a column as index helper to facilitate the function that compare the similarity of each drug name
- We'll take deeper dive into the drug name has a strong similarity
- Drop duplicated drug name based on similarity comparison
- Clean the companies name, some have their name in the datasets in different format

In [69]:
# import text similarity tool
import difflib
import jellyfish

In [70]:
# group drug names by company level
co_phase_drug = df1_industry[['sponsor_name','drug_name']].drop_duplicates()
co_phase_drug

,sponsor_name,drug_name
0,"T3D Therapeutics, Inc.",t3d-959
1,"Genentech, Inc.",[18f]gtp1
3,"Otsuka Pharmaceutical Co., Ltd.",2
4,"Otsuka Pharmaceutical Co., Ltd.",3
5,"Otsuka Pharmaceutical Co., Ltd.",6
...,...,...
186,TauRx Therapeutics Ltd,trx0237
190,Merck Sharp & Dohme Corp.,verubecestat
191,"VIVUS, Inc.",vi-1121
192,EIP Pharma Inc,vx-745


Before we do. anything, we are going to drop rows where length of the name only has 3 or less than 3 letters

In [71]:
co_phase_drug[co_phase_drug['drug_name'].str.len()<=3]

,sponsor_name,drug_name
3,"Otsuka Pharmaceutical Co., Ltd.",2
4,"Otsuka Pharmaceutical Co., Ltd.",3
5,"Otsuka Pharmaceutical Co., Ltd.",6
42,Charsire Biotechnology Corp.,bac
107,Wyeth is now a wholly owned subsidiary of Pfizer,gsi


In [72]:
co_phase_drug.drop(co_phase_drug[co_phase_drug['drug_name'].str.len()<=3].index, inplace=True)

In [73]:
# create index helper to facilate lookup function
co_phase_drug['ix_helper'] = co_phase_drug['sponsor_name'] + co_phase_drug['drug_name']

In [74]:
# create function to find similar name
def count_similar_name(ix_helper):
    try:
    #     print(ix_helper)
        company_name = co_phase_drug[co_phase_drug['ix_helper'] == ix_helper]['sponsor_name'].values[0]
        # create drug list by company name to compare with each name within the company's trials
        drug_list = list(co_phase_drug[co_phase_drug['sponsor_name'] == company_name]['drug_name'])
        # the drug we are going to compare the list
        drug = co_phase_drug[co_phase_drug['ix_helper'] == ix_helper]['drug_name'].values[0]
        # remove the drug name we are going to compare with
        drug_list.remove(drug)
        return int(len(difflib.get_close_matches(drug, drug_list, cutoff=0.7)))
    except IndexError:
        return ''

In [75]:
# create function to find similar name
def similar_name(ix_helper):
    try:
    #     print(ix_helper)
        company_name = co_phase_drug[co_phase_drug['ix_helper'] == ix_helper]['sponsor_name'].values[0]
        # create drug list by company name to compare with each name within the company's trials
        drug_list = list(co_phase_drug[co_phase_drug['sponsor_name'] == company_name]['drug_name'])
        # the drug we are going to compare the list
        drug = co_phase_drug[co_phase_drug['ix_helper'] == ix_helper]['drug_name'].values[0]
        # remove the drug name we are going to compare with
        drug_list.remove(drug)
        return difflib.get_close_matches(drug, drug_list, cutoff=0.7)
    except IndexError:
        return ''

In [76]:
# apply the function to df
co_phase_drug['count_similar_name'] = co_phase_drug['ix_helper'].apply(count_similar_name)
co_phase_drug['similar_name'] = co_phase_drug['ix_helper'].apply(similar_name)

We'll start to look into these rows have ***1 similar drug name***. It looks like they would either   
- include a sign in the name, example: masitinib (ab1010)  
- or it has sort of description, example: [cor388 capsule]  
So, our solution is to keep the one has the shorter name. For example, we'll keep 'bryostatin' instead of 'bryostatin 1'  
If they are same length, we'll keep those because majority of those are different drugs

In [77]:
# df for rows have one similar name
similar_name_one = co_phase_drug[co_phase_drug['count_similar_name']==1]

In [78]:
# finding the length difference betwenn comparable name and drug name
diff_len = (similar_name_one['similar_name'].apply(lambda x: len(x[0])) 
            - similar_name_one['drug_name'].apply(lambda x: len(x)) )
# the one has the longer name
similar_name_one[diff_len<0]

,sponsor_name,drug_name,ix_helper,count_similar_name,similar_name
15,Pfizer,acc-001+,Pfizeracc-001+,1,[acc-001]
65,"Neurotrope Bioscience, Inc.",bryostatin 1,"Neurotrope Bioscience, Inc.bryostatin 1",1,[bryostatin]
82,FORUM Pharmaceuticals Inc,drug: evp-6124,FORUM Pharmaceuticals Incdrug: evp-6124,1,[evp-6124]
151,ACADIA Pharmaceuticals Inc.,pimavanserin tartrate,ACADIA Pharmaceuticals Inc.pimavanserin tartrate,1,[pimavanserin]


Looks like we can drop all of them, because those are duplicates, they just inputed differently

In [79]:
# drop those rows
co_phase_drug.drop(similar_name_one[diff_len<0].index, inplace=True)

In [80]:
co_phase_drug[co_phase_drug['count_similar_name'] == 2]

,sponsor_name,drug_name,ix_helper,count_similar_name,similar_name
28,Otsuka Pharmaceutical Development & Commercial...,aripiprazole,Otsuka Pharmaceutical Development & Commercial...,2,"[brexpiprazole, brexpiprazole,]"
59,Otsuka Pharmaceutical Development & Commercial...,brexpiprazole,Otsuka Pharmaceutical Development & Commercial...,2,"[brexpiprazole,, aripiprazole]"
62,Otsuka Pharmaceutical Development & Commercial...,"brexpiprazole,",Otsuka Pharmaceutical Development & Commercial...,2,"[brexpiprazole, aripiprazole]"
98,FORUM Pharmaceuticals Inc,evp-6124,FORUM Pharmaceuticals Incevp-6124,2,"[evp-0962, drug: evp-6124]"
177,"FUJIFILM Toyama Chemical Co., Ltd.",t-817ma,"FUJIFILM Toyama Chemical Co., Ltd.t-817ma",2,"[t-817ma-l, t-817ma-h]"
179,"FUJIFILM Toyama Chemical Co., Ltd.",t-817ma-h,"FUJIFILM Toyama Chemical Co., Ltd.t-817ma-h",2,"[t-817ma-l, t-817ma]"
180,"FUJIFILM Toyama Chemical Co., Ltd.",t-817ma-l,"FUJIFILM Toyama Chemical Co., Ltd.t-817ma-l",2,"[t-817ma-h, t-817ma]"


We'll manually drop these rows because not all of them are duplicates, we tried to play with cutoff parameter but we couldn't find the best one

In [81]:
co_phase_drug.drop(co_phase_drug[co_phase_drug['drug_name']=='brexpiprazole,'].index, inplace = True)
co_phase_drug.drop(co_phase_drug[co_phase_drug['drug_name']=='t-817ma-h'].index, inplace = True)
co_phase_drug.drop(co_phase_drug[co_phase_drug['drug_name']=='t-817ma-l'].index, inplace = True)

We fixed many drug names, we'll clean the dataframe first, we'll only keep sponsor names and drug names and drop duplicates

In [82]:
co_phase_drug = co_phase_drug[['sponsor_name', 'drug_name']]

We'll start to cosolidate entities into one because some have their name differently multiple times, see the example below

In [83]:
co_phase_drug[co_phase_drug['sponsor_name'].str.contains('Pfizer')]['sponsor_name'].value_counts()

Pfizer                                              8
Wyeth is now a wholly owned subsidiary of Pfizer    2
Name: sponsor_name, dtype: int64

We are going to only look at the first word of the sponsor name. We'll use the first word to find out other names companies used. it works for the majority of cases

In [84]:
# split name
sponsor_name_df = co_phase_drug[['sponsor_name']].drop_duplicates()
sponsor_name_df['short_name'] = [re.split(r'\s', name)[0] for name in sponsor_name_df['sponsor_name']]

In [85]:
# create a df to save all other names used for each short name
short_sponsor_name = sponsor_name_df[['short_name']].drop_duplicates()
short_sponsor_name['names_used'] = [list(sponsor_name_df[sponsor_name_df['sponsor_name'].str.contains(shor_name)]['sponsor_name'].values) 
 for shor_name in short_sponsor_name['short_name']]

In [86]:
short_sponsor_name['number_of_names'] = short_sponsor_name['names_used'].apply(lambda x: len(x))

As we can seee below, there are few cases we need adjust the name manually. 
- Abbvie is split from Abbott, so all trials should be under Abbview
- There are two Mercks, there are not the same one
- Chase should be under Allergan based on its description
- There two companies have 'High' in its name
- Short name 'H' doesn't really make sense, we need to look into it



In [87]:
short_sponsor_name[short_sponsor_name['number_of_names']>1]

,short_name,names_used,number_of_names
6,Pfizer,"[Pfizer, Wyeth is now a wholly owned subsidiar...",2
7,AbbVie,"[AbbVie, AbbVie (prior sponsor, Abbott)]",2
9,Abbott,"[Abbott, AbbVie (prior sponsor, Abbott)]",2
17,Eisai,"[Eisai Inc., Eisai Limited, Eisai Co., Ltd.]",3
28,Otsuka,[Otsuka Pharmaceutical Development & Commercia...,2


In [88]:
# we are going to drop Abbott from this df, later we'll use this as a lookup table 
short_sponsor_name.drop(short_sponsor_name[short_sponsor_name['short_name']=='Abbott'].index, inplace=True)
short_sponsor_name.drop(short_sponsor_name[short_sponsor_name['short_name']=='Wyeth'].index, inplace=True)
short_sponsor_name.drop(short_sponsor_name[short_sponsor_name['short_name']=='Chase'].index, inplace=True)


In [89]:
# there are some manually added rows above dont have number of names used, we are going to add manually
short_sponsor_name.fillna(1, inplace=True)

In [90]:
# covert list into string to facilita our look up 
for row in range(len(short_sponsor_name)):
    lt = short_sponsor_name.iloc[row, 1]
    string = ''
    for i in range(len(lt)):
        string += ' '+lt[i]
#     print(string)
    short_sponsor_name.iloc[row, 1] = string

### Final Clean

In [91]:
co_phase_drug[co_phase_drug['drug_name'].str.contains('drug')]

,sponsor_name,drug_name
81,Eisai Inc.,drug:
84,Humanetics Corporation,drug: nic5-15


In [92]:
# can't modify the name by one regex
co_phase_drug['drug_name'] = [re.sub(r'^drug:?\s?[-]?','', i) for i in co_phase_drug['drug_name']]
co_phase_drug['drug_name'] = [re.sub(r'^drug','', i) for i in co_phase_drug['drug_name']]

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [93]:
co_phase_drug[co_phase_drug['drug_name'].str.contains('active')]

,sponsor_name,drug_name
16,Novartis Pharmaceuticals,active
17,Eisai Inc.,active
18,Cognition Therapeutics,active treatment- ct1812


In [94]:
# can't modify the name by one regex
co_phase_drug['drug_name'] = [re.sub(r'^active\s?', '',i) for i in co_phase_drug['drug_name']]
co_phase_drug['drug_name'] = [re.sub(r'treatment-?', '',i) for i in co_phase_drug['drug_name']]

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [95]:
# drop these row, these doesn't make sense. double checked with original dataset
co_phase_drug.drop(co_phase_drug[co_phase_drug['drug_name'].str.contains('normal')].index, inplace=True)
co_phase_drug.drop(co_phase_drug[co_phase_drug['drug_name'].str.contains('experimental')].index, inplace=True)
# these are the one we fixed above
co_phase_drug.drop(co_phase_drug[co_phase_drug['drug_name'] == ''].index, inplace=True)

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [96]:
short_sponsor_name[short_sponsor_name.fillna(1)['number_of_names']>1]

,short_name,names_used,number_of_names
6,Pfizer,Pfizer Wyeth is now a wholly owned subsidiary...,2
7,AbbVie,"AbbVie AbbVie (prior sponsor, Abbott)",2
17,Eisai,"Eisai Inc. Eisai Limited Eisai Co., Ltd.",3
28,Otsuka,Otsuka Pharmaceutical Development & Commercia...,2


In [97]:
# majority have their name in one format
short_sponsor_name['number_of_names'].value_counts()

1    74
2     3
3     1
Name: number_of_names, dtype: int64

In [98]:
# our final step is to re-map the names company filled
def name_unify(filled_name):
#     print(filled_name)
    num_names = (short_sponsor_name[short_sponsor_name['names_used'].str.contains(filled_name, regex=False)])['number_of_names'].values
    # if companies have multiple names used then re-map that to short name
    if num_names > 1:
        return (short_sponsor_name[short_sponsor_name['names_used'].str.contains(filled_name, regex=False)])['short_name'].values[0]
    else:
        return (short_sponsor_name[short_sponsor_name['names_used'].str.contains(filled_name, regex=False)])['names_used'].values[0]
        

In [99]:
co_phase_drug['fixed_name'] = co_phase_drug['sponsor_name'].apply(lambda x: name_unify(x))

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


The drug names and company names look good now, next we are going to add its latest phase, nct_id and latest completion date into the dataframe

### Adding Key Info into The Main Dataframe
- nct_id
- start date
- completion date
- phase
- Overall Status

In [100]:
co_phase_drug.reset_index(drop=True, inplace=True)
co_phase_drug.head()

,sponsor_name,drug_name,fixed_name
0,"T3D Therapeutics, Inc.",t3d-959,"T3D Therapeutics, Inc."
1,"Genentech, Inc.",[18f]gtp1,"Genentech, Inc."
2,Pfizer,aab-003,Pfizer
3,AbbVie,abbv-8e12,AbbVie
4,Abbott,abt-089,AbbVie


In [104]:
trial_ids = []
start_dates = []
completion_date = []
trial_phase = []
overall_status = []
# use for loop to extract key info
for ix in co_phase_drug.index:
    drug_name = co_phase_drug.loc[ix, 'drug_name']
    sponsor_name = co_phase_drug.loc[ix, 'sponsor_name']
    temp_df = df1_industry[(df1_industry['drug_name'].str.contains(drug_name, regex=False)) & 
                      (df1_industry['sponsor_name'].str.contains(sponsor_name, regex=False))]
    max_completion_date = temp_df['primary_completion_date'].max()
    
    try:
        phase = temp_df[temp_df['primary_completion_date'] == max_completion_date]['phase'].values[0]
        start_date = temp_df[temp_df['primary_completion_date'] == max_completion_date]['study_first_submitted_date'].values[0]
        trial_id = temp_df[temp_df['primary_completion_date'] == max_completion_date]['nct_id'].values[0]
        status = temp_df[temp_df['primary_completion_date'] == max_completion_date]['overall_status'].values[0]
    # there are few de-regesitered trials so it doesn't have completion date
    except IndexError:
        phase = temp_df['phase'].values[0]
        start_date = temp_df['study_first_submitted_date'].values[0]
        trial_id = temp_df['nct_id'].values[0]
        status = temp_df['overall_status'].values[0]
        print(drug_name, sponsor_name, max_completion_date, phase, start_date, trial_id, status)
    # add into list
    trial_ids.append(trial_id)
    start_dates.append(start_date)
    completion_date.append(max_completion_date)
    trial_phase.append(phase)
    overall_status.append(status)

triglyceride Cerecin nan Phase 2/Phase 3 2010-09-24 NCT01211782
jnj-39393406 Janssen Pharmaceutica N.V., Belgium nan Phase 1 2010-06-03 NCT01137799


In [105]:
# add into dataframe
co_phase_drug['nct_id'] = trial_ids
co_phase_drug['start_date'] = start_dates
co_phase_drug['completion_date'] = completion_date
co_phase_drug['phase'] = trial_phase
co_phase_drug['overall_status'] = overall_status

/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jonzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [106]:
co_phase_drug

,sponsor_name,drug_name,fixed_name,nct_id,start_date,completion_date,phase,overall_status
0,"T3D Therapeutics, Inc.",t3d-959,"T3D Therapeutics, Inc.",NCT04251182,2020-01-24,2021-08-13,Phase 2,Not yet recruiting
1,"Genentech, Inc.",[18f]gtp1,"Genentech, Inc.",NCT03828747,2019-01-29,2021-05-07,Phase 2,Recruiting
2,Pfizer,aab-003,Pfizer,NCT01193608,2010-08-19,2013-10-31,Phase 1,Completed
3,AbbVie,abbv-8e12,AbbVie,NCT03712787,2018-10-18,2026-08-11,Phase 2,Enrolling by invitation
4,Abbott,abt-089,AbbVie,NCT00555204,2007-11-07,2009-06-30,Phase 2,Terminated
...,...,...,...,...,...,...,...,...
123,TauRx Therapeutics Ltd,trx0237,TauRx Therapeutics Ltd,NCT03446001,2018-02-08,2021-12-31,Phase 3,Recruiting
124,Merck Sharp & Dohme Corp.,verubecestat,Merck Sharp & Dohme Corp.,NCT01953601,2013-09-25,2018-04-17,Phase 3,Terminated
125,"VIVUS, Inc.",vi-1121,"VIVUS, Inc.",NCT01428362,2011-08-31,2013-08-31,Phase 2,Completed
126,EIP Pharma Inc,vx-745,EIP Pharma Inc,NCT02423200,2015-04-03,2016-09-30,Phase 2,Completed


In [107]:
co_phase_drug.to_csv('Alzheimer Commercial Trials.csv', index=False)